In [2]:
#3
import numpy as np
from Bio import SeqIO, Align
from Bio.PDB import *
import warnings
from Bio import BiopythonWarning

# Suppress Biopython warnings
warnings.simplefilter('ignore', BiopythonWarning)

class ProteinStructurePredictor:
    def __init__(self):
        # Initialize basic amino acid properties
        self.aa_properties = {
            'A': {'hydrophobicity': 1.8, 'volume': 88.6, 'polarity': 0},
            'R': {'hydrophobicity': -4.5, 'volume': 173.4, 'polarity': 1},
            'N': {'hydrophobicity': -3.5, 'volume': 114.1, 'polarity': 1},
            # ... (other amino acids would be defined similarly)
        }
        # Initialize secondary structure propensities
        self.ss_propensities = {
            'A': {'helix': 1.42, 'sheet': 0.83, 'coil': 0.9},
            'R': {'helix': 1.21, 'sheet': 0.93, 'coil': 0.99},
            'N': {'helix': 0.67, 'sheet': 0.89, 'coil': 1.33},
            # ... (other amino acids would be defined similarly)
        }

    def search_template_database(self, query_sequence):
        """
        Simulate searching for template structures in the PDB database
        Returns: mock template data for demonstration
        """
        # Mock template structure
        mock_template = {
            'pdb_id': '1ABC',
            'sequence': 'SIMILAR_SEQUENCE',
            'resolution': 2.5,
            'coordinates': np.random.rand(len(query_sequence), 3)
        }
        return mock_template

    def align_sequences(self, query_sequence, template_sequence):
        """
        Perform sequence alignment between query and template
        """
        aligner = Align.PairwiseAligner()
        aligner.open_gap_score = -10
        aligner.extend_gap_score = -0.5
        alignments = aligner.align(query_sequence, template_sequence)
        return alignments[0]

    def predict_secondary_structure(self, sequence):
        """
        Predict secondary structure using propensities
        """
        structure = []
        window_size = 5
        padded_seq = 'X' * (window_size // 2) + sequence + 'X' * (window_size // 2)
        
        for i in range(len(sequence)):
            window = padded_seq[i:i + window_size]
            helix_score = sheet_score = coil_score = 0

            # Calculate propensity scores
            for aa in window:
                if aa in self.ss_propensities:
                    helix_score += self.ss_propensities[aa]['helix']
                    sheet_score += self.ss_propensities[aa]['sheet']
                    coil_score += self.ss_propensities[aa]['coil']

            # Assign structure based on highest score
            if max(helix_score, sheet_score, coil_score) == helix_score:
                structure.append('H')
            elif max(helix_score, sheet_score, coil_score) == sheet_score:
                structure.append('E')
            else:
                structure.append('C')
        
        return ''.join(structure)

    def build_backbone(self, sequence, secondary_structure):
        """
        Build basic backbone structure using predicted secondary structure
        Returns: numpy array of coordinates
        """
        coordinates = np.zeros((len(sequence), 3))
        current_pos = np.array([0.0, 0.0, 0.0])
        
        for i in range(len(sequence)):
            if secondary_structure[i] == 'H':  # Helix
                current_pos += np.array([1.5, 0.5, 0.5])
            elif secondary_structure[i] == 'E':  # Sheet
                current_pos += np.array([1.5, 0.0, 0.0])
            else:  # Coil
                current_pos += np.array([1.0, np.random.rand(), np.random.rand()])
            coordinates[i] = current_pos
        
        return coordinates

    def refine_structure(self, coordinates):
        """
        Simple structure refinement
        """
        # Apply basic energy minimization (simplified)
        refined_coords = coordinates + np.random.normal(0, 0.1, coordinates.shape)
        return refined_coords

    def predict_structure(self, sequence):
        """
        Main method to predict protein structure
        """
        # Step 1: Search for template
        template = self.search_template_database(sequence)
        
        # Step 2: Align sequences
        alignment = self.align_sequences(sequence, template['sequence'])
        
        # Step 3: Predict secondary structure
        secondary_structure = self.predict_secondary_structure(sequence)
        
        # Step 4: Build initial backbone
        initial_coords = self.build_backbone(sequence, secondary_structure)
        
        # Step 5: Refine structure
        final_coords = self.refine_structure(initial_coords)
        
        return {
            'coordinates': final_coords,
            'secondary_structure': secondary_structure,
            'alignment': alignment,
            'template_used': template['pdb_id']
        }

    def save_structure(self, coordinates, sequence, filename):
        """
        Save predicted structure in PDB format
        """
        structure = Structure.Structure('predicted')
        model = Model.Model(0)
        chain = Chain.Chain('A')
        
        for i, (coord, aa) in enumerate(zip(coordinates, sequence)):
            residue = Residue.Residue((' ', i, ' '), aa, '')
            atom = Atom.Atom('CA', coord, 20.0, 1.0, ' ', 'CA', i, 'C')
            residue.add(atom)
            chain.add(residue)
        
        model.add(chain)
        structure.add(model)
        
        io = PDBIO()
        io.set_structure(structure)
        io.save(filename)

def main():
    # Example usage
    predictor = ProteinStructurePredictor()
    
    # Example sequence (shortened for demonstration)
    sequence = "MKWVTFISLLLLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKALVLIAFAQYLQQCP"
    
    # Predict structure
    result = predictor.predict_structure(sequence)
    
    # Save structure
    predictor.save_structure(
        result['coordinates'],
        sequence,
        'predicted_structure.pdb'
    )
    
    print(f"Secondary structure prediction: {result['secondary_structure']}")
    print(f"Template used: {result['template_used']}")
    print("Structure has been saved to 'predicted_structure.pdb'")

if __name__ == "__main__":
    main()

Secondary structure prediction: HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCHHHHHHHHHHHHHHH
Template used: 1ABC
Structure has been saved to 'predicted_structure.pdb'
